In [11]:
from web3 import Web3, HTTPProvider
import json
import rlp
from eth_typing import HexStr
from eth_utils import to_bytes
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [21]:
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545',request_kwargs={'timeout': 600}))
w3.is_connected()
len(w3.eth.accounts)

251

In [3]:
accounts = w3.eth.accounts
keys = json.load(open('./keys.json'))['private_keys']
AbiPath = './build/contracts/'
AMMabi = json.load(open(AbiPath + 'CPAMM.json'))['abi']
Energyabi = json.load(open(AbiPath + 'EnergyToken.json'))['abi']
Moneyabi = json.load(open(AbiPath + 'MoneyToken.json'))['abi']
Testabi = json.load(open(AbiPath + 'Test.json'))['abi']

MoneyToken_address = Web3.to_checksum_address("0xe78A0F7E598Cc8b0Bb87894B0F60dD2a88d6a8Ab")
EnergyToken_address = Web3.to_checksum_address("0x5b1869D9A4C187F2EAa108f3062412ecf0526b24")
Amm_address = Web3.to_checksum_address("0xC89Ce4735882C9F0f0FE26686c53074E09B0D550")

AMMContract = w3.eth.contract(address=Amm_address,abi=AMMabi)
MoneyContract = w3.eth.contract(address=MoneyToken_address,abi=Moneyabi)
EnergyContract = w3.eth.contract(address=EnergyToken_address,abi=Energyabi)
Test = w3.eth.contract(address="0x254dffcd3277C0b1660F6d42EFbB754edaBAbC2B",abi = Testabi)

admin = Web3.to_checksum_address(w3.eth.accounts[0])


In [4]:
indexes = pd.read_excel("EC_EV_dataset.xlsx",sheet_name ="Indexes")
indexes.set_index("Player ID",inplace=True)

loads = pd.read_excel("EC_EV_dataset.xlsx",sheet_name ="Load")
loads = loads.add_prefix("load")
loads = loads.rename(columns={"loadTime": "Time"})
loads = pd.wide_to_long(loads,stubnames="load",i= "Time",j='ID')

pv = pd.read_excel("EC_EV_dataset.xlsx",sheet_name="PV")
pv = pv.add_prefix("pv_gen")
pv = pv.rename(columns={"pv_genTime": "Time"})
pv = pd.wide_to_long(pv,stubnames="pv_gen",i= "Time",j='ID')

sell_price = pd.read_excel("EC_EV_dataset.xlsx",sheet_name="Sell Price")
sell_price = sell_price.add_prefix("sell_price")
sell_price = sell_price.rename(columns={"sell_priceTime": "Time"})
sell_price = pd.wide_to_long(sell_price,stubnames="sell_price",i= "Time",j='ID')

buy_price = pd.read_excel("EC_EV_dataset.xlsx",sheet_name="Buy Price")
buy_price = buy_price.add_prefix("buy_price")
buy_price = buy_price.rename(columns={"buy_priceTime": "Time"})
buy_price = pd.wide_to_long(buy_price,stubnames="buy_price",i= "Time",j='ID')

data = loads.merge(right= pv,left_index=True,right_index=True).merge(right=buy_price,right_index=True,left_index=True).merge(right=sell_price,right_index=True,left_index=True)



In [23]:
# 
data["balance_of_energy"] = np.nan
data["balance_of_money"] =np.nan

k_upper = data['buy_price'].agg('mean')
k_lower = data['sell_price'].agg('mean')
for i in range(1,97):
    for j in range (1,251):
        load = data.loc[(i,j),"load"]
        pv_gen = data.loc[(i,j),"pv_gen"]
        address = accounts[j]
        #print(address)
        #if member needs energy
        print(str(i) + " " + str(j))
        if (load-pv_gen > 0):
            # Fund account
            Funds = int((load-pv_gen)*k_upper*10e18)
            MoneyContract.functions.transfer(address,Funds).transact({'from':admin})
            # approve
            MoneyContract.functions.approve(Amm_address,Funds).transact({'from':address})
            # swap
            AMMContract.functions.swap(MoneyToken_address,Funds).transact({'from':address})
        else: 
            # Fund Energy
            Surplus = int((pv_gen - load)*10e18)
            EnergyContract.functions.transfer(address,Surplus).transact({'from':admin})
            # approve
            EnergyContract.functions.approve(Amm_address,Surplus).transact({'from':address})
            # swap
            AMMContract.functions.swap(EnergyToken_address,Surplus).transact({'from':address})
    
    for j in range (1,250):
        address = accounts[j]
        data.loc[(i,j),'balance_of_energy'] = EnergyContract.functions.balanceOf(address).call()
        data.loc[(i,j),'balance_of_money'] = MoneyContract.functions.balanceOf(address).call()

            
            


            

        

1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
1 50
1 51
1 52
1 53
1 54
1 55
1 56
1 57
1 58
1 59
1 60
1 61
1 62
1 63
1 64
1 65
1 66
1 67
1 68
1 69
1 70
1 71
1 72
1 73
1 74
1 75
1 76
1 77
1 78
1 79
1 80
1 81
1 82
1 83
1 84
1 85
1 86
1 87
1 88
1 89
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
1 100
1 101
1 102
1 103
1 104
1 105
1 106
1 107
1 108
1 109
1 110
1 111
1 112
1 113
1 114
1 115
1 116
1 117
1 118
1 119
1 120
1 121
1 122
1 123
1 124
1 125
1 126
1 127
1 128
1 129
1 130
1 131
1 132
1 133
1 134
1 135
1 136
1 137
1 138
1 139
1 140
1 141
1 142
1 143
1 144
1 145
1 146
1 147
1 148
1 149
1 150
1 151
1 152
1 153
1 154
1 155
1 156
1 157
1 158
1 159
1 160
1 161
1 162
1 163
1 164
1 165
1 166
1 167
1 168
1 169
1 170
1 171
1 172
1 173
1 174
1 175
1 176
1 177
1 178
1 179
1 180
1 181
1 182
1 183
1 184
1 18

ReadTimeout: HTTPConnectionPool(host='127.0.0.1', port=8545): Read timed out. (read timeout=30.0)

In [12]:
MoneyContract.all_functions()

[<Function allowance(address,address)>,
 <Function approve(address,uint256)>,
 <Function balanceOf(address)>,
 <Function decimals()>,
 <Function name()>,
 <Function owner()>,
 <Function symbol()>,
 <Function totalSupply()>,
 <Function transfer(address,uint256)>,
 <Function transferFrom(address,address,uint256)>]

In [5]:
k_upper = loads['price'].agg('mean')

In [74]:
pd.wide_to_long(loads,stubnames="load",i= "loadTime",j='ID')

,,load
loadTime,ID,
1,1,1.556923
2,1,1.430336
3,1,1.304366
4,1,1.218622
5,1,0.792239
...,...,...
92,250,1.291791
93,250,1.255507
94,250,1.252617


In [63]:
loads.isna().any().any()

np.False_

In [36]:
pd.wide_to_long(loads,"load",i= "ID",j='time')

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
for index,row in indexes.iterrows():
    

In [148]:
#Add 50 Liquidity to AMM
MoneyContract.functions.approve(Amm_address,50*10**18).transact({"from":me,"nonce": w3.eth.get_transaction_count(me),"gas":2000000,"gasPrice":w3.eth.gas_price})
EnergyContract.functions.approve(Amm_address,50*10**18).transact({"from":me,"nonce": w3.eth.get_transaction_count(me),"gas":2000000,"gasPrice":w3.eth.gas_price})

AMMContract.functions.addLiquidity(50*10**18,50*10**18).transact({"from":me,"nonce": w3.eth.get_transaction_count(me),"gas":2000000,"gasPrice":w3.eth.gas_price})

HexBytes('0x39c8263cdd8bceb7acdcc31e09f7b7a5baed64e87653cacf3df1b11b64b2aaad')

In [151]:
MoneyContract.functions.approve(Amm_address,50*10**18).transact({"from":me,"nonce": w3.eth.get_transaction_count(me),"gas":2000000,"gasPrice":w3.eth.gas_price})
#Swap Energy to Money
AMMContract.functions.swap(MoneyToken_address,20*10**18).transact({"from":me,"nonce": w3.eth.get_transaction_count(me),"gas":2000000,"gasPrice":w3.eth.gas_price})

HexBytes('0x70e767b08f58dd0beeb6ddc24a910d3b8acdaa45c57b368be5346a8b9faf0d40')

In [144]:
MoneyContract.functions.approve("0x254dffcd3277C0b1660F6d42EFbB754edaBAbC2B",5*10**18).transact({"from":me,"nonce": w3.eth.get_transaction_count(me),"gas":2000000,"gasPrice":w3.eth.gas_price})

HexBytes('0x274658551076f44eb41cfbfa2a7aafd8b78252a14e54a90b24f851a76c7d016e')

In [145]:
x = Test.functions.send(5 * 10 ** 18).transact({"from":me,"nonce": w3.eth.get_transaction_count(me),"gas":2000000,"gasPrice":w3.eth.gas_price})

In [147]:
MoneyContract.functions.balanceOf("0x254dffcd3277C0b1660F6d42EFbB754edaBAbC2B").call()

5000000000000000000

In [138]:
MoneyContract.functions.transferFrom(me,accounts[1],3*10**18).transact({"from":me,"nonce": w3.eth.get_transaction_count(me),"gas":2000000,"gasPrice":w3.eth.gas_price})

HexBytes('0x923cae33742b7694490d5925fe6bc94137b89b27c40a1f99c94a520a3c155713')

In [121]:
5 * 10**18

5000000000000000000

In [123]:
Test.functions.send(5 * 10**18).transact({"from":me,"nonce": w3.eth.get_transaction_count(me),"gas":2000000,"gasPrice":w3.eth.gas_price})

Test.functions.add

ABIFunctionNotFound: ("The function 'send' was not found in this contract's abi.", ' Are you sure you provided the correct contract abi?')

In [152]:
print("Money: " + str(MoneyContract.functions.balanceOf(accounts[0]).call()) )
print("Energy: " + str(EnergyContract.functions.balanceOf(me).call()) )
print("AMM: "+ str(AMMContract.functions.get_reserve0().call()))

Money: 6999925000000000000000000
Energy: 6999969940000000000000000
AMM: 70000000000000000000


In [111]:
x= AMMContract.functions.addLiquidity(3000000000000000000,3000000000000000000).transact({"from":me,"nonce": w3.eth.get_transaction_count(me),"gas":2000000,"gasPrice":w3.eth.gas_price})

In [112]:
y = AMMContract.functions.swap(MoneyToken_address,3 * 10**decimals).transact({"from":me,"nonce": w3.eth.get_transaction_count(me),"gas":2000000,"gasPrice":w3.eth.gas_price})

In [95]:
AMMContract.functions.totalSupply().call()

0

In [88]:
w3.eth.get_transaction_receipt(x)


AttributeDict({'transactionHash': HexBytes('0xbd6153cd8d4943b87a50428dd623c7b039b55b9ede10ba1e0e68ae32604bc6ef'),
 'transactionIndex': 0,
 'blockNumber': 4,
 'blockHash': HexBytes('0x9c9086ca91576d777bb1134f17f062a9267c5894721f8801660dd927e47ee0c9'),
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xCfEB869F69431e42cdB54A4F4f105C19C080A601',
 'cumulativeGasUsed': 29054,
 'gasUsed': 29054,
 'contractAddress': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 0

In [40]:
#MoneyContract.functions.transfer(accounts[1],7 * (10 ** decimal)).stransact({"from":me})
txs = rlp.decode(hex_to_bytes("0xf8a910850684ee180082e48694a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4880b844a9059cbb000000000000000000000000b8b59a7bc828e6074a4dd00fa422ee6b92703f9200000000000000000000000000000000000000000000000000000000010366401ba0e2a4093875682ac6a1da94cdcc0a783fe61a7273d98e1ebfe77ace9cab91a120a00f553e48f3496b7329a7c0008b3531dd29490c517ad28b0e6c1fba03b79a1dee"), x)
txs.to_dict()

NameError: name 'to_bytes' is not defined

In [72]:
AMMContract.functions

AttributeError: 'Contract' object has no attribute 'methods'

In [13]:
MoneyToken_address = Web3.to_checksum_address("0xe78A0F7E598Cc8b0Bb87894B0F60dD2a88d6a8Ab")
EnergyToken_address = Web3.to_checksum_address("0x5b1869D9A4C187F2EAa108f3062412ecf0526b24")
Amm = Web3.to_checksum_address("0xC89Ce4735882C9F0f0FE26686c53074E09B0D550")

def swapMoney(address,amount):
    
    AMMContract.functions.swap(MoneyToken_address,amount)

In [5]:
# Get Contract
contract_address =Web3.to_checksum_address("0x5fbdb2315678afecb367f032d93f642f64180aa3")
contract = w3.eth.contract(address=contract_address,abi = abi)

In [14]:
contract = w3.eth.contract(address=Amm,abi = abi)